In [16]:
#necessary imports:
import geopandas as gpd
import pandas as pd
import random
import sys
sys.path.append('../')
import modules.predict as predict
import modules.encoder as encoder
import copy


import numpy as np

In [3]:
gdf = gpd.read_file('./import/TEST_BEFORE_COMPLETION.geojson')

In [4]:
#before performing the mapping we want to fill incomplete columns 
target_columns = ['numberOfFloors_original', 'rooftype_original']
selected_predictors = ['class', 'first_year', 'last_year', 'numberOfFloors_original', 'material.value', 'rooftype_original']
gdf_new = copy.deepcopy(gdf)
for target in target_columns:
    filled_column, is_pred_column, comments_col = predict.fill_missing_values(target, gdf, selected_predictors)
    #gdf[target] = filled_column
    #put the filled column in the gdf substituting the original one
    gdf_new[target] = filled_column
    gdf_new[target + '_type'] = is_pred_column
    gdf_new[target + '_comments'] = comments_col


# fill missing values should be called also during the life of the model
# it should keep track of the used model
encoder should consider all the mapping fields
encoder should consider ranges defined as an extraroot property of the model
extraroot properties should be listed in the csv
every field of the extension should be documented


In [15]:
#merge the comments column with the one in the original gdf


In [5]:
# specify in the mapping which columns should change name. 
mapping = {
    'type': 'class',
    'time.estimatedStart.year': 'first_year',    
    'time.estimatedEnd.year': 'last_year',  
    'numberOfFloors.value':'numberOfFloors_original',
    'numberOfFloors.paradata.type':'numberOfFloors_original_type',
    'roof.type.value': 'rooftype_original',
    'roof.type.paradata.type': 'rooftype_original_type'
}
# on the left, the name of the field according to the Historical CityJSON extension. On the right, the name of the field in the input geodata.
# The fields that are already correctly encoded don't need to be inserted in the mapping
# it's not necessary to eliminate non mappable fields, when creating the cityjson they will be ignored

# Map the GeoDataFrame to HistoricalCityJSON
mapped_gdf = encoder.map_gdf_fields_to_historicalcityjson(gdf_new, mapping)

mapped_gdf['roof.type.value'].unique()


array(['gabled', 'flat', 'cupola', 'destroyed building'], dtype=object)

In [7]:
gdf= mapped_gdf
for row in gdf.iterrows():
    if row[1]['roof.type.value'] == 'gabled' or row[1]['roof.type.value'] == 'slanted':
        row[1]['roof.type.value'] = random.choice(['hip','gable'])
        gdf.loc[row[0],'roof.type.value'] = row[1]['roof.type.value']
    if row[1]['numberOfFloors.value'] == '1-3':
        row[1]['numberOfFloors.value'] = random.choice([1,2,3])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']
    if row[1]['numberOfFloors.value'] == '4+':
        row[1]['numberOfFloors.value'] = random.choice([4,5,6])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']
    if row[1]['numberOfFloors.value'] == '6+':
        row[1]['numberOfFloors.value'] = random.choice([6,7,8])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']


mapping_type = {'archway':'Archway'}
mapping_rooftype = {
    'destroyed building': '',
    'cupola':'domed',
                    }


In [8]:
df_mapped_values = encoder.map_column_values(gdf, 'type', mapping_type)
df_mapped_values = encoder.map_column_values(df_mapped_values, 'roof.type.value', mapping_rooftype)

In [9]:
df_mapped_values['roof.type.value'].unique()

array(['hip', 'flat', 'gable', 'domed', ''], dtype=object)

In [20]:
#save the dataframe as a geojson
df_mapped_values.to_file('./import/TEST_AFTER_COMPLETION.geojson', driver='GeoJSON')

In [10]:
fields_df = pd.read_csv('../extension/geojson_mapping.csv')
fields_df
# we want to iterate over each row and take the value before the . in a list (only unique, since many elements share the first part before the dot)





,fields
0,type
1,height.value
2,height.sources.n.name
3,height.sources.n.type
4,height.sources.n.notes
...,...
143,time.estimatedEnd.sources.n.notes
144,time.estimatedEnd.paradata.type
145,time.estimatedEnd.paradata.author
146,time.estimatedEnd.paradata.date


In [13]:
print(fields_df)

                                             fields
0                                              type
1                                      height.value
2                             height.sources.n.name
3                             height.sources.n.type
4                            height.sources.n.notes
..                                              ...
143    time.estimatedEnd.timeMoment.sources.n.notes
144      time.estimatedEnd.timeMoment.paradata.type
145    time.estimatedEnd.timeMoment.paradata.author
146      time.estimatedEnd.timeMoment.paradata.date
147  time.estimatedEnd.timeMoment.paradata.comments

[148 rows x 1 columns]


In [11]:
import csv
fields_df = pd.read_csv('../extension/geojson_mapping.csv')
def extract_unique_values(fields_df):
    unique_values = set()
    for field in fields_df['fields']:
        if '.value' in field:
            value = field.split('.value')[0]
            unique_values.add(value)
        elif '.sources' in field:
            value = field.split('.sources')[0]
            unique_values.add(value)
        elif '.paradata' in field:
            value = field.split('.paradata')[0]
            unique_values.add(value)

    return list(unique_values)

unique_values = extract_unique_values(fields_df)
print(unique_values)

['roof.parameters.railingHeight', 'floorHeight', 'roof.parameters.gableSides', 'numberOfFloors', 'roof.parameters.upperFloorThickness', 'time.estimatedStart', 'time.appearances.n', 'roof.parameters.baseFloorThickness', 'height', 'parcels.n', 'roof.type', 'roof.parameters.domePercentBaseRadius', 'time.estimatedEnd', 'roof.parameters.domePercentVertRadius', 'roof.parameters.slope', 'roof.parameters.eavesOverhang', 'roof.parameters.railingWidth', 'material']


In [15]:
value = unique_values[1].split('.')

['floorHeight']

In [13]:
unique_values[2].split('.')

['roof', 'parameters', 'gableSides']

In [14]:
time.estimatedEnd.paradata.comments

NameError: name 'time' is not defined